In [1]:
# Import dependencies
from datetime import datetime
from datetime import date
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Read into excel file
xls = pd.ExcelFile("./Resources/KPMG_VI_New_raw_data_update_final.xlsx")
transactions_xls = pd.read_excel(xls, "Transactions")
new_customer_list_xls = pd.read_excel(xls, "NewCustomerList")
customer_demographic_xls = pd.read_excel(xls, "CustomerDemographic")
customer_address_xls = pd.read_excel(xls, "CustomerAddress")

In [5]:
# Set first row as header
customer_demographic_xls.columns = customer_demographic_xls.iloc[0]

# Drop unwanted row
customer_demographic_xls = customer_demographic_xls.drop(0)

# Set customer_id as index
#customer_demographic_xls = customer_demographic_xls.set_index("customer_id")

# View customer demographic dataframe
customer_demographic_xls

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,default,owns_car,tenure
1,1,Laraine,Medendorp,F,93,1953-10-12 00:00:00,Executive Secretary,Health,Mass Customer,N,"""'",Yes,11
2,2,Eli,Bockman,Male,81,1980-12-16 00:00:00,Administrative Officer,Financial Services,Mass Customer,N,<script>alert('hi')</script>,Yes,16
3,3,Arlin,Dearle,Male,61,1954-01-20 00:00:00,Recruiting Manager,Property,Mass Customer,N,2018-02-01 00:00:00,Yes,15
4,4,Talbot,NaN,Male,33,1961-10-03 00:00:00,NaN,IT,Mass Customer,N,() { _; } >_[$($())] { touch /tmp/blns.shellsh...,No,7
5,5,Sheila-kathryn,Calton,Female,56,1977-05-13 00:00:00,Senior Editor,NaN,Affluent Customer,N,NIL,Yes,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3996,3996,Rosalia,Halgarth,Female,8,1975-08-09 00:00:00,VP Product Management,Health,Mass Customer,N,-100,No,19
3997,3997,Blanch,Nisuis,Female,87,2001-07-13 00:00:00,Statistician II,Manufacturing,High Net Worth,N,â¦testâ§,Yes,1
3998,3998,Sarene,Woolley,U,60,NaN,Assistant Manager,IT,High Net Worth,N,NaN,No,NaN
3999,3999,Patrizius,NaN,Male,11,1973-10-24 00:00:00,NaN,Manufacturing,Affluent Customer,N,Â¡â¢Â£Â¢âÂ§Â¶â¢ÂªÂºââ,Yes,10


In [ ]:
# Check for null values
customer_demographic_xls.count()

In [ ]:
# Drop null values
customer_demographic_xls = customer_demographic_xls.dropna()

In [ ]:
customer_demographic_xls.count()

In [ ]:
customer_demographic_xls = customer_demographic_xls.reset_index()

In [ ]:
customer_demographic_xls

In [ ]:
customer_demographic_xls["gender"].value_counts()

In [ ]:
# Replace "Femal" and "F" with "Female" for value consistency
customer_demographic_xls["gender"] = customer_demographic_xls["gender"].replace(["F", "Femal"], "Female")

In [ ]:
customer_demographic_xls["gender"].value_counts()

In [ ]:
customer_demographic_xls.drop(columns="default")

In [ ]:
# Adding an age column to know each customer's ages
def calculate_age(born):
    born = datetime.strptime(born, "%Y.%m.%d").date()
    today = date.today()
    return today.year - born.year - ((today.month, today.day) -  (born.month, born.day))

customer_demographic_xls["age"] = customer_demographic_xls["DOB"].apply(calculate_age)